In [1]:
### Start the run from here Shivani

In [2]:
pip install transformers

In [3]:
# pip install tensorflow-text

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [32]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
from transformers import TFBertForSequenceClassification, BertTokenizer

In [34]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.transformer.layer.1.ffn.lin1.weight', 'distilbert.transformer.layer.1.attention.v_lin.bias', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.1.attention.out_lin.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight

In [35]:

# drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()


In [36]:
df = pd.read_csv('Cleaned_Resumes_n_lab_limit_1500.csv')
df.head()

,cleaned_txt,lab,cleaned_lab
0,database administrator database administrator ...,Database_Administrator,[1 0 0 0 0 0 0 0 0 0]
1,database administrator sql server database adm...,Database_Administrator,[1 0 0 0 0 0 0 0 0 0]
2,oracle database administrator oracle oracle da...,Database_Administrator,[1 0 0 0 0 0 0 0 0 0]
3,amazon redshift administrator etl developer bu...,Database_Administrator,[1 0 0 0 0 0 0 0 0 0]
4,scrum master scrum master scrum master richmon...,Database_Administrator,[1 0 0 0 0 0 0 0 0 0]


In [37]:
import ast


df['cleaned_lab'] = df['cleaned_lab'].apply(lambda x: x.replace(' ',','))
df['cleaned_lab'] = df['cleaned_lab'].apply(lambda x: ast.literal_eval(x))

In [38]:
df

,cleaned_txt,lab,cleaned_lab
0,database administrator database administrator ...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,database administrator sql server database adm...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,oracle database administrator oracle oracle da...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,amazon redshift administrator etl developer bu...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,scrum master scrum master scrum master richmon...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
25815,wordpress developer web designer freelancer wo...,"Web_Developer ,Software_Developer","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"
25816,web developer designer designer web developer ...,"Web_Developer ,Software_Developer ,Front_End_D...","[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]"
25817,android developer android android developer hc...,"Web_Developer ,Software_Developer","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"
25818,web developer los angeles ca work experience w...,"Web_Developer ,Software_Developer","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"


In [39]:
from tqdm.auto import tqdm
import tensorflow as tf

In [40]:
X_input_ids = np.zeros((len(df), 512))
X_attn_masks = np.zeros((len(df), 512))

In [41]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['cleaned_txt'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = 512,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [42]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [43]:
X_input_ids

array([[  101.,  7809.,  8911., ..., 29296.,  8241.,   102.],
       [  101.,  7809.,  8911., ...,     0.,     0.,     0.],
       [  101., 14721.,  7809., ...,     0.,     0.,     0.],
       ...,
       [  101., 11924.,  9722., ...,  2478., 19842.,   102.],
       [  101.,  4773.,  9722., ...,     0.,     0.,     0.],
       [  101.,  5034.,  4007., ...,  6628.,  3930.,   102.]])

In [44]:
y = np.vstack(df['cleaned_lab'].values)
y.shape

(25820, 10)

In [45]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, y))
dataset.take(1) # one sample data

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [46]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [47]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset


In [48]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [49]:
#run from here
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [50]:
dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 10), dtype=tf.int64, name=None))>

In [51]:
p = 0.8
train_size = int((len(df)//16)*p)

In [52]:

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)


In [53]:
for layer in model.layers:
    layer.trainable = False

In [54]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
# intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(10, activation='sigmoid', name='output_layer')(bert_embds) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 512, 7                                     

In [55]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [56]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [57]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    batch_size = 32,
    epochs=4
)

Epoch 1/4
1290/1290 [==============================] - 1081s 831ms/step - loss: 0.5157 - accuracy: 0.3138 - val_loss: 0.4150 - val_accuracy: 0.5433
Epoch 2/4
1290/1290 [==============================] - 1077s 835ms/step - loss: 0.4387 - accuracy: 0.4421 - val_loss: 0.4070 - val_accuracy: 0.5457
Epoch 3/4
1290/1290 [==============================] - 1076s 834ms/step - loss: 0.4338 - accuracy: 0.4415 - val_loss: 0.4046 - val_accuracy: 0.5364
Epoch 4/4
1290/1290 [==============================] - 1076s 834ms/step - loss: 0.4334 - accuracy: 0.4411 - val_loss: 0.4084 - val_accuracy: 0.5457


# Custome model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


import pandas as pd
import numpy as np
import ast

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Cleaned_Resumes_n_lab_limit_1500.csv')
df['cleaned_lab'] = df['cleaned_lab'].apply(lambda x: x.replace(' ',','))
df['cleaned_lab'] = df['cleaned_lab'].apply(lambda x: ast.literal_eval(x))
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,cleaned_txt,lab,cleaned_lab
0,database administrator database administrator ...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,database administrator sql server database adm...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,oracle database administrator oracle oracle da...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,amazon redshift administrator etl developer bu...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,scrum master scrum master scrum master richmon...,Database_Administrator,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df['cleaned_txt'],df['cleaned_lab'], test_size=0.2, random_state=42)

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index) + 1

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
max_len = 1500  # Set the maximum length of a sequence
x_train = pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_len, padding='post')

In [ ]:
print(x_train.shape)
print(x_test.shape)

(20656, 1500)
(5164, 1500)


In [ ]:
y_train = np.vstack(y_train)
y_test = np.vstack(y_test)

print(y_test.shape)
print(y_train.shape)

(5164, 10)
(20656, 10)


In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
class MultiLabelTransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, num_labels, d_model=256, num_heads=4, ff_dim=4, dropout=0.2, max_len=512):
        super(MultiLabelTransformerModel, self).__init__()

        # Embedding layer
        self.embedding = layers.Embedding(vocab_size, d_model, input_length=max_len)

        # Multi-Head Self Attention layer
        # self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.attention_heads = [layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads) for _ in range(3)] # loop is for adding more MultiHeadAttention layers

        # Feedforward layer
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dropout(dropout),
            layers.Dense(d_model)
        ])

        # Global average pooling layer
        self.global_average_pooling = layers.GlobalAveragePooling1D()

        # Dense layers for classification
        self.dense1 = layers.Dense(512, activation='relu')
        self.dropout = layers.Dropout(dropout)
        self.dense2 = layers.Dense(num_labels, activation='sigmoid')  # Binary classification

    def call(self, inputs):
        x = self.embedding(inputs)

        # Multi-Head Self Attention
        attention_outputs = [attention_head(x, x) for attention_head in self.attention_heads]
        x = layers.Add()(attention_outputs)

        # Feedforward layer
        x = self.ffn(x)

        x = self.global_average_pooling(x)
        x = self.dense1(x)
        x = self.dropout(x)
        output = self.dense2(x)
        return output

# Model with 2 MultiHeadAttention

In [ ]:
num_labels = 10  # Assuming binary classification
model = MultiLabelTransformerModel(vocab_size=vocab_size, num_labels=num_labels, max_len=max_len)

learning_rate = 0.001  # Replace with your desired learning rate
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs= 10 , batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
 72/646 [==>...........................] - ETA: 5:00 - loss: 0.4688 - accuracy: 0.4466

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

162/162 [==============================] - 26s 163ms/step


In [ ]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.79      0.85       608
           1       0.95      0.94      0.94      2454
           2       0.80      0.71      0.75      1213
           3       0.73      0.78      0.76       873
           4       0.80      0.78      0.79       560
           5       0.76      0.68      0.71      1128
           6       0.74      0.80      0.77       884
           7       0.86      0.83      0.85       707
           8       0.77      0.78      0.77       364
           9       0.89      0.82      0.85       387

   micro avg       0.83      0.81      0.82      9178
   macro avg       0.82      0.79      0.80      9178
weighted avg       0.84      0.81      0.82      9178
 samples avg       0.86      0.85      0.83      9178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save('/content/drive/My Drive/custom_transformer')

# Model with 3 MultiHeadAttention

In [ ]:
num_labels = 10  # Assuming binary classification
model_2 = MultiLabelTransformerModel(vocab_size=vocab_size, num_labels=num_labels, max_len=max_len)

learning_rate = 0.001  # Replace with your desired learning rate
optimizer = Adam(learning_rate=learning_rate)

model_2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
model_2.fit(x_train, y_train, epochs= 10 , batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
646/646 [==============================] - 414s 624ms/step - loss: 0.3176 - accuracy: 0.6037 - val_loss: 0.1904 - val_accuracy: 0.8048
Epoch 2/10
646/646 [==============================] - 407s 630ms/step - loss: 0.1674 - accuracy: 0.8266 - val_loss: 0.1677 - val_accuracy: 0.7843
Epoch 3/10
646/646 [==============================] - 402s 622ms/step - loss: 0.1396 - accuracy: 0.8498 - val_loss: 0.1575 - val_accuracy: 0.8137
Epoch 4/10
646/646 [==============================] - 401s 622ms/step - loss: 0.1102 - accuracy: 0.8737 - val_loss: 0.1742 - val_accuracy: 0.8002
Epoch 5/10
646/646 [==============================] - 407s 630ms/step - loss: 0.0802 - accuracy: 0.8862 - val_loss: 0.2028 - val_accuracy: 0.8156
Epoch 6/10
646/646 [==============================] - 407s 630ms/step - loss: 0.0597 - accuracy: 0.8953 - val_loss: 0.2144 - val_accuracy: 0.8021
Epoch 7/10
  9/646 [..............................] - ETA: 5:56 - loss: 0.0401 - accuracy: 0.8750

KeyboardInterrupt: ignored

In [ ]:
y_pred2 = model_2.predict(x_test)
y_pred2 = (y_pred2 > 0.5).astype(int)
y_pred2

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]])

In [ ]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82       608
           1       0.96      0.93      0.95      2454
           2       0.76      0.81      0.79      1213
           3       0.73      0.74      0.74       873
           4       0.82      0.75      0.78       560
           5       0.70      0.79      0.74      1128
           6       0.79      0.74      0.76       884
           7       0.84      0.83      0.84       707
           8       0.81      0.67      0.73       364
           9       0.90      0.75      0.82       387

   micro avg       0.83      0.82      0.82      9178
   macro avg       0.82      0.78      0.80      9178
weighted avg       0.83      0.82      0.82      9178
 samples avg       0.85      0.85      0.82      9178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_2.save('/content/drive/My Drive/custom_transformer2(3multheadattention)')